In [1]:
# Import all dependencies

import numpy as np
import os
from netCDF4 import Dataset
from numpy import sum 

In [2]:
# Specify the CMIP7 dataset

INPUT4MIPS_DIR = "/g/data/qv56/replicas/input4MIPs"
EXT_FILE = "CMIP7/CMIP/uoexeter/UOEXETER-CMIP-2-0-0/atmos/monC/ext/gnz/v20250227/ext_input4MIPs_aerosolProperties_CMIP_UOEXETER-CMIP-2-0-0_gnz_185001-202112-clim.nc"
EXT_PATH = os.path.join(INPUT4MIPS_DIR, EXT_FILE)

In [3]:
# Read in the CMIP7 dataset

ext_set = Dataset(EXT_PATH, mode="r")

In [4]:
# Check that we have 12 months of data

times = ext_set["time"][:]
MONTHS = 12.0
assert len(times) == MONTHS

In [5]:
# Create weights corresponding to the proportion of days in each month
MONTH_WEIGHTS = np.array([31, 28.2425, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]) / 365.2425
assert(sum(MONTH_WEIGHTS) == 1.0)

In [6]:
# Check that the difference between latitudes is 5 degrees

lats = ext_set["lat"][:]
LAT_DIFF = 5.0
for j in range(1, len(lats)):
    assert lats[j] - lats[j-1] == LAT_DIFF

In [7]:
# Check that the height of every atmospheric layer is 500 metres

heights = ext_set["height"][:]
LAYER_HEIGHT = 500.0
for j in range(1, len(heights)):
    assert heights[j] - heights[j-1] == LAYER_HEIGHT

In [8]:
# Check that wavelength[7] is 550 nm as prescribed

wavelengths = ext_set["wavelength"][:]
WAVELENGTH = 550.0 * 1e-9
WAVELENGTH_INDEX = wavelengths.searchsorted([WAVELENGTH])
assert wavelengths[WAVELENGTH_INDEX] == WAVELENGTH

In [9]:
# Create weights corresponding to the relative area of each latitude band

sindeg = lambda t: np.sin(np.radians(t))
lat_weights = [(sindeg(lat + LAT_DIFF / 2.0) - sindeg(lat - LAT_DIFF / 2.0)) / 2.0 for lat in lats]
assert sum(lat_weights) == 1.0

In [10]:
# Select the ext values for the prescribed wavelength

ext_var = ext_set["ext"]
exts_at_wl = np.nan_to_num(ext_var[:, :, :, WAVELENGTH_INDEX]).reshape(np.shape(ext_var)[:-1])

In [11]:
# Calculate the saod values for each month and latitude band by quadrature of ext over the heights of the atmosphere layers 

ext_dims = ext_var.dimensions
EXT_HEIGHT_AXIS = ext_dims.index("height")
saods_at_wl = sum(exts_at_wl, axis=EXT_HEIGHT_AXIS) * LAYER_HEIGHT

In [12]:
# Take the time average of the saod values for each latitude band

weighted_saod_at_each_lat = np.dot(MONTH_WEIGHTS, saods_at_wl)

In [13]:
# Calculate the average saod by weighting by the relative area of each latitude band

weighted_saod = np.dot(lat_weights, weighted_saod_at_each_lat)
print(weighted_saod)

0.014116564638800556


In [14]:
# Calculate the naive average saod value by equally wighting each latitude band

avg_saod = sum(weighted_saod_at_each_lat) / len(lats)
print(avg_saod)

0.01587605431569429
